In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image

In [2]:
query = """
(SELECT 
PLAYER_NAME, Season, "Regular Season" as Category, SUM(G) GP, SUM(MIN)MIN, SUM(CONTESTED_SHOTS)CONTESTED_SHOTS, SUM(CONTESTED_SHOTS_2PT)CONTESTED_SHOTS_2PT, SUM(CONTESTED_SHOTS_3PT)CONTESTED_SHOTS_3PT, SUM(DEFLECTIONS)DEFLECTIONS, SUM(CHARGES_DRAWN)CHARGES_DRAWN, SUM(SCREEN_ASSISTS)SCREEN_ASSISTS, SUM(SCREEN_AST_PTS)SCREEN_AST_PTS, SUM(OFF_LOOSE_BALLS_RECOVERED)OFF_LOOSE_BALLS_RECOVERED, SUM(DEF_LOOSE_BALLS_RECOVERED)DEF_LOOSE_BALLS_RECOVERED, SUM(LOOSE_BALLS_RECOVERED)LOOSE_BALLS_RECOVERED, SUM(OFF_BOXOUTS)OFF_BOXOUTS, SUM(DEF_BOXOUTS) DEF_BOXOUTS, SUM(BOX_OUT_PLAYER_TEAM_REBS)BOX_OUT_PLAYER_TEAM_REBS, SUM(BOX_OUT_PLAYER_REBS)BOX_OUT_PLAYER_REBS, SUM(BOX_OUTS) BOX_OUTS
FROM `perceptive-ivy-290216.nba_api_hustle_stats.player_hustle_total` 
GROUP BY 1,2
ORDER BY 1,2)
UNION ALL
(SELECT 
PLAYER_NAME, Season, "Playoffs" as Category, SUM(G) GP, SUM(MIN)MIN, SUM(CONTESTED_SHOTS)CONTESTED_SHOTS, SUM(CONTESTED_SHOTS_2PT)CONTESTED_SHOTS_2PT, SUM(CONTESTED_SHOTS_3PT)CONTESTED_SHOTS_3PT, SUM(DEFLECTIONS)DEFLECTIONS, SUM(CHARGES_DRAWN)CHARGES_DRAWN, SUM(SCREEN_ASSISTS)SCREEN_ASSISTS, SUM(SCREEN_AST_PTS)SCREEN_AST_PTS, SUM(OFF_LOOSE_BALLS_RECOVERED)OFF_LOOSE_BALLS_RECOVERED, SUM(DEF_LOOSE_BALLS_RECOVERED)DEF_LOOSE_BALLS_RECOVERED, SUM(LOOSE_BALLS_RECOVERED)LOOSE_BALLS_RECOVERED, SUM(OFF_BOXOUTS)OFF_BOXOUTS, SUM(DEF_BOXOUTS) DEF_BOXOUTS, SUM(BOX_OUT_PLAYER_TEAM_REBS)BOX_OUT_PLAYER_TEAM_REBS, SUM(BOX_OUT_PLAYER_REBS)BOX_OUT_PLAYER_REBS, SUM(BOX_OUTS) BOX_OUTS
FROM `perceptive-ivy-290216.nba_api_hustle_stats.playoffs_player_hustle_total`
GROUP BY 1,2
ORDER BY 1,2)
"""
project_id = "perceptive-ivy-290216"
hustle_totals = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
query = """
SELECT PLAYER_NAME, TEAM_ABBREVIATION, SEASON FROM `perceptive-ivy-290216.player_team_info.player_bios_regs_totals`
# WHERE season='2023-24'
"""
project_id = "perceptive-ivy-290216"
player_team = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    
player_team['color'] = player_team['TEAM_ABBREVIATION'].apply(team_color)

In [5]:
#REGULAR SEASON

In [6]:
#TOTALS

In [7]:
year="2023-24"

In [32]:
#Combine player team data with player aggregated data so that we can color based on team
hustle_totals_year=hustle_totals[hustle_totals["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["PLAYER_NAME","Season"], right_on=["PLAYER_NAME","SEASON"])
#Filter to regular season
hustle_season=hustle_totals_year[(hustle_totals_year["Category"]=='Regular Season')]
#Create a player and team dict for color for future coloring
player_color_dict_shot_dist=dict(zip(hustle_season['PLAYER_NAME'], hustle_season['color']))
team_color_dict_shot_dist=dict(zip(hustle_season['TEAM_ABBREVIATION'], hustle_season['color']))

In [41]:
#Remove _ from Column Names
hustle_season.columns = hustle_season.columns.str.replace('_', ' ') 
hustle_season.head()

,PLAYER NAME,Season,Category,GP,MIN,CONTESTED SHOTS,CONTESTED SHOTS 2PT,CONTESTED SHOTS 3PT,DEFLECTIONS,CHARGES DRAWN,...,DEF LOOSE BALLS RECOVERED,LOOSE BALLS RECOVERED,OFF BOXOUTS,DEF BOXOUTS,BOX OUT PLAYER TEAM REBS,BOX OUT PLAYER REBS,BOX OUTS,TEAM ABBREVIATION,SEASON,color
214,Jusuf Nurkic,2023-24,Regular Season,76,2078.0,568,484,84,166,1,...,24,59,97,164,248,171,261,PHX,2023-24,#e56020
215,Moussa Diabate,2023-24,Regular Season,11,64.0,18,12,6,5,0,...,1,3,6,2,7,6,8,LAC,2023-24,#0C2340
216,Luka Samanic,2023-24,Regular Season,43,404.0,63,25,38,17,1,...,0,3,6,3,9,7,9,UTA,2023-24,#0C2340
217,Talen Horton-Tucker,2023-24,Regular Season,51,1009.0,139,59,80,85,0,...,14,28,3,7,9,4,10,UTA,2023-24,#0C2340
218,Russell Westbrook,2023-24,Regular Season,68,1529.0,132,73,59,124,5,...,28,55,9,7,15,11,16,LAC,2023-24,#0C2340


In [46]:
category="CONTESTED SHOTS"

In [48]:
fig_hustle_season = px.bar(
  hustle_season.sort_values(by=category,ascending=False).head(25),
  x="PLAYER NAME", 
  y=category,
  color="PLAYER NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player {} For The {} Season</b>".format(category.replace('_', ' ').capitalize(),year),
  hover_data=['PLAYER NAME', 'Season', 'Category', 'GP', 'MIN', 'CONTESTED SHOTS',
       'CONTESTED SHOTS 2PT', 'CONTESTED SHOTS 3PT', 'DEFLECTIONS',
       'CHARGES DRAWN', 'SCREEN ASSISTS', 'SCREEN AST PTS',
       'OFF LOOSE BALLS RECOVERED', 'DEF LOOSE BALLS RECOVERED',
       'LOOSE BALLS RECOVERED', 'OFF BOXOUTS', 'DEF BOXOUTS',
       'BOX OUT PLAYER TEAM REBS', 'BOX OUT PLAYER REBS', 'BOX OUTS',
       'TEAM ABBREVIATION', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_hustle_season.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_hustle_season

In [ ]:
save_year=2024

In [ ]:
fig_hustle_season.write_html("/Users/rdesh723/statpulse-html/plots/Hustle/{}/Player {} For The {} Season.html".format(save_year,category.replace('_', ' ').capitalize(),year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [49]:
category=['CONTESTED SHOTS',
       'CONTESTED SHOTS 2PT', 'CONTESTED SHOTS 3PT', 'DEFLECTIONS',
       'CHARGES DRAWN', 'SCREEN ASSISTS', 'SCREEN AST PTS',
       'OFF LOOSE BALLS RECOVERED', 'DEF LOOSE BALLS RECOVERED',
       'LOOSE BALLS RECOVERED', 'OFF BOXOUTS', 'DEF BOXOUTS',
       'BOX OUT PLAYER TEAM REBS', 'BOX OUT PLAYER REBS', 'BOX OUTS']
for i in category:
    fig_hustle_season = px.bar(
    hustle_season.sort_values(by=i,ascending=False).head(25),
    x="PLAYER NAME", 
    y=i,
    color="PLAYER NAME",
    color_discrete_map=player_color_dict_shot_dist,     
    template='plotly_white',
    title="<b>Player {} For The {} Season</b>".format(i.replace('_', ' ').capitalize(),year),
    hover_data=['PLAYER NAME', 'Season', 'Category', 'GP', 'MIN', 'CONTESTED SHOTS',
       'CONTESTED SHOTS 2PT', 'CONTESTED SHOTS 3PT', 'DEFLECTIONS',
       'CHARGES DRAWN', 'SCREEN ASSISTS', 'SCREEN AST PTS',
       'OFF LOOSE BALLS RECOVERED', 'DEF LOOSE BALLS RECOVERED',
       'LOOSE BALLS RECOVERED', 'OFF BOXOUTS', 'DEF BOXOUTS',
       'BOX OUT PLAYER TEAM REBS', 'BOX OUT PLAYER REBS', 'BOX OUTS',
       'TEAM ABBREVIATION', 'SEASON'], 
    height=800, 
    width=1200,
    )
    fig_hustle_season.update_layout(
        margin=dict(l=5, r=5, t=40, b=5),
        showlegend=False,
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_hustle_season.write_html("/Users/rdesh723/statpulse-html/plots/Hustle/{}/Player {} For The {} Season.html".format(save_year,i.replace('_', ' ').capitalize(),year),full_html=False, include_plotlyjs='cdn')